In [291]:
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import scipy.optimize as sco
from functools import reduce
import pandas as pd


In [292]:
#Q1

#Set number of assets and obersations
n_assets = 20
n_obs = 60
np.random.seed(1)
#generate random normal rates
return_vec = np.random.normal(0.025,0.015,(n_obs,n_assets))
return_vec = np.asmatrix(return_vec)

#Create matrix for QP
P = opt.matrix(np.cov(return_vec, rowvar=False))
q = opt.matrix(0.0, (n_assets ,1))

#Merge G
G = -opt.matrix(np.eye(return_vec.shape[0],return_vec.shape[1]))
#Weight
Gw = np.transpose(opt.matrix(1.0, (n_assets ,1)))
G_sum = opt.matrix(np.concatenate((G,-Gw),axis=0))


h = opt.matrix(0.0, (len(return_vec) + 1 ,1))
#weight constraint
#h1 = opt.matrix(1.0)
#h_sum = np.concatenate((h,-h1),axis=0)

#Merge A
r = opt.matrix(np.mean(return_vec, axis=0))
g = np.transpose(opt.matrix(1.0, (n_assets ,1)))
A = np.concatenate((g,r),axis=0)
A = opt.matrix(A)

#Merge b interest rates and sum of weights = 1
rf = 0.02
x = 0.001
g1 = opt.matrix(1.0)
r1 = opt.matrix(rf+x)
b = np.concatenate((g1,r1),axis=0)
b = opt.matrix(b)
#solution
sol1 = solvers.qp(P,q,G_sum,h,A,b)['x']
#print(sol1['x'])
print(sol1)


     pcost       dcost       gap    pres   dres
 0:  2.9118e-05 -2.6882e+00  1e+02  9e+00  2e+01
 1:  2.9643e-05 -1.2290e+00  2e+01  1e+00  2e+00
 2:  5.8864e-05  2.4612e+00  8e+00  5e-01  1e+00
 3:  1.8850e-04  4.8770e+00  8e+00  4e-01  8e-01
 4:  2.5577e-04  1.8473e+01  5e+00  3e-01  6e-01
 5:  2.6022e-04  2.0477e+02  9e+00  3e-01  6e-01
 6:  2.6105e-04  8.0579e+03  1e+01  3e-01  6e-01
 7:  2.6104e-04  5.7554e+06  7e+02  3e-01  6e-01
 8:  2.6106e-04  1.0587e+09  1e+03  3e-01  6e-01
 9:  2.3917e-06  1.4659e+14  3e+14  5e-01  1e+00
10:  1.3040e-04  3.0494e+14  1e+13  5e-01  2e+00
11:  1.3211e-04  4.7060e+15  9e+13  5e-01  1e+02
12:  1.3324e-04  2.3667e+17  5e+14  5e-01  3e+03
13:  1.3340e-04  1.0037e+20  7e+15  5e-01  1e+06
14:  1.3340e-04  1.7832e+21  1e+15  5e-01  3e+07
15:  3.3867e-06  3.6152e+25  8e+25  5e-01  4e+12
16:  1.1313e-05  1.2557e+25  9e+25  4e-01  3e+12
17:  4.0521e-05  1.9071e+25  9e+25  3e-01  2e+12
18:  6.8096e-05  1.3840e+26  7e+25  2e-01  2e+12
19:  8.8065e-05  6.47

In [300]:
sol1 = solvers.qp(P,q,G_sum,h,A,b)['x']

#calculating variance using t(weight)*sigma matrix * weight
t_sol1 = np.transpose(sol1)
v=t_sol1.dot(P)
var = v.dot(sol1)
print("variance of Sol1 is:", var)

     pcost       dcost       gap    pres   dres
 0:  2.4730e-03  2.0937e+01  4e+02  2e+01  3e+01
 1:  4.5168e-03  5.2886e+01  9e+01  5e+00  7e+00
 2:  1.0523e-02  2.2990e+02  1e+02  5e+00  7e+00
 3:  1.5638e-02  9.1776e+02  1e+02  5e+00  7e+00
 4:  1.6673e-02  1.1555e+04  3e+02  5e+00  7e+00
 5:  1.6941e-02  6.1400e+05  7e+02  5e+00  7e+00
 6:  1.6947e-02  5.3792e+08  9e+04  5e+00  7e+00
 7:  1.1169e-02  4.7802e+15  2e+14  3e+00  6e+00
 8:  1.7989e-02  5.7305e+15  2e+14  4e+00  5e+00
Terminated (singular KKT matrix).
variance of Sol1 is: [[0.03597746]]


In [302]:
#Target Excess return from 0 to 5% by 0.1% increase
rx = np.arange(0.0,0.05,0.001)
ratio = []
va = []
#Use for loop to see how ratio of variance changes as Excess Return increases
for r in rx:
    print(r)
    r1 = opt.matrix(rf+r)
    b = np.concatenate((g1,r1),axis=0)
    b = opt.matrix(b)
    sol1 = solvers.qp(P, q, G_sum, h, A, b)['x']
    
    #calculate variance
    t_sol1 = np.transpose(sol1)
    v=t_sol1.dot(P)
    var = v.dot(sol1)
    va.append(var)
    ratio.append(r/var)
    


0.0
     pcost       dcost       gap    pres   dres
 0:  4.1695e-05 -2.5549e+00  1e+02  9e+00  2e+01
 1:  4.3131e-05 -8.1423e-01  2e+01  1e+00  2e+00
 2:  9.5872e-05  4.4378e+00  1e+01  7e-01  1e+00
 3:  2.9420e-04  9.5690e+00  1e+01  6e-01  1e+00
 4:  3.8433e-04  4.1541e+01  8e+00  5e-01  9e-01
 5:  3.9001e-04  5.7688e+02  2e+01  4e-01  8e-01
 6:  3.9090e-04  2.9231e+04  4e+01  4e-01  8e-01
 7:  3.9088e-04  2.7106e+07  2e+03  4e-01  8e-01
 8:  7.1617e-06  8.6307e+12  2e+13  4e-01  8e-01
 9:  2.9453e-05  6.1533e+12  2e+13  4e-01  7e-01
10:  9.6259e-05  1.4001e+13  1e+13  2e-01  5e-01
11:  1.3513e-04  4.9310e+13  1e+13  2e-01  4e-01
12:  1.5160e-04  2.0025e+14  2e+13  2e-01  8e-01
13:  1.5417e-04  1.1925e+15  7e+13  1e-01  7e+00
14:  1.5639e-04  1.6669e+16  3e+14  1e-01  6e+01
15:  1.5738e-04  8.6034e+17  2e+15  1e-01  5e+03
16:  1.5749e-04  4.4367e+20  2e+16  1e-01  2e+06
17:  1.5750e-04  1.1675e+22  6e+15  1e-01  1e+08
18:  4.7099e-06  1.7378e+27  2e+27  6e-01  1e+13
19:  1.9059e-05  

12:  3.7863e-05 -2.5387e+13  7e+13  2e-01  5e-01
13:  1.0722e-04 -3.2976e+13  7e+13  1e-01  2e-01
14:  1.5241e-04  2.2248e+13  4e+13  8e-02  4e-01
15:  1.5575e-04  2.4192e+14  6e+13  7e-02  2e+00
16:  1.2665e-04  1.2163e+15  1e+14  7e-02  1e+01
17:  1.5028e-04  3.9073e+15  3e+14  7e-02  2e+01
18:  1.5429e-04  3.7080e+16  1e+15  7e-02  3e+02
19:  1.5554e-04  1.0406e+18  6e+15  7e-02  1e+04
20:  1.5579e-04  1.7834e+20  5e+16  7e-02  2e+06
21:  1.5582e-04  1.4814e+22  4e+16  7e-02  2e+08
22:  3.9186e-05  1.3400e+27  5e+27  4e-01  1e+13
23:  3.7833e-05  1.3095e+27  5e+27  4e-01  1e+13
24:  3.8107e-05  8.0625e+26  5e+27  4e-01  2e+13
25:  3.9820e-05 -2.6469e+27  5e+27  4e-01  2e+13
26:  4.7971e-05 -2.0425e+28  6e+27  4e-01  6e+13
27:  7.9697e-05 -9.8996e+28  1e+28  4e-01  2e+14
28:  1.3752e-04 -3.7737e+29  6e+28  3e-01  1e+15
29:  6.8005e-05 -3.5014e+30  1e+30  2e-01  5e+15
30:  9.7442e-05 -5.8794e+30  5e+30  9e-02  9e+15
31:  9.8460e-05 -5.6370e+30  5e+30  9e-02  9e+15
32:  1.0113e-04 -4.4

     pcost       dcost       gap    pres   dres
 0:  2.8253e-04 -3.1713e-02  2e+02  1e+01  2e+01
 1:  3.4739e-04  4.8457e+00  3e+01  2e+00  3e+00
 2:  8.6985e-04  2.2982e+01  2e+01  2e+00  2e+00
 3:  1.2644e-03  1.0029e+02  3e+01  1e+00  2e+00
 4:  1.9228e-03  4.1633e+02  5e+01  1e+00  2e+00
 5:  1.9251e-03  3.7868e+03  1e+02  1e+00  2e+00
 6:  1.9199e-03  1.0495e+05  5e+02  1e+00  2e+00
 7:  1.9189e-03  1.9968e+07  6e+03  1e+00  2e+00
 8:  1.5902e-04  1.4628e+14  4e+13  3e-01  5e-01
Terminated (singular KKT matrix).
0.021
     pcost       dcost       gap    pres   dres
 0:  3.2178e-04  3.8114e-01  2e+02  1e+01  2e+01
 1:  4.0370e-04  5.6813e+00  3e+01  2e+00  3e+00
 2:  1.0679e-03  2.6916e+01  2e+01  2e+00  3e+00
 3:  1.5974e-03  1.2477e+02  3e+01  2e+00  2e+00
 4:  2.1977e-03  5.7818e+02  5e+01  2e+00  2e+00
 5:  2.1956e-03  6.3080e+03  2e+02  1e+00  2e+00
 6:  2.1895e-03  2.2198e+05  8e+02  1e+00  2e+00
 7:  2.1885e-03  6.0419e+07  1e+04  1e+00  2e+00
 8:  5.1504e-05  2.0685e+14  3e

75:  4.6108e-04  1.4983e+78  2e+73  7e-01  8e+62
Terminated (singular KKT matrix).
0.032
     pcost       dcost       gap    pres   dres
 0:  9.2461e-04  6.1898e+00  2e+02  1e+01  2e+01
 1:  1.4066e-03  1.8355e+01  5e+01  3e+00  5e+00
 2:  3.0548e-03  6.8736e+01  5e+01  3e+00  4e+00
 3:  6.0437e-03  3.2445e+02  6e+01  3e+00  4e+00
 4:  6.2151e-03  2.4728e+03  1e+02  3e+00  4e+00
 5:  6.3221e-03  6.2180e+04  2e+02  3e+00  4e+00
 6:  6.3253e-03  1.9939e+07  1e+04  3e+00  4e+00
 7:  6.3269e-03  2.7572e+10  3e+05  3e+00  4e+00
 8:  6.3269e-03  6.7267e+10  6e+03  3e+00  4e+00
 9:  2.0554e-04  7.4605e+15  6e+15  3e+00  3e+01
10:  1.4894e-03  1.6872e+16  4e+15  2e+00  2e+01
11:  1.2036e-03  7.9419e+16  8e+15  2e+00  3e+01
12:  1.6433e-03  6.4596e+17  3e+16  2e+00  5e+02
13:  1.7290e-03  9.3772e+18  2e+17  2e+00  6e+03
14:  1.7802e-03  4.4058e+20  1e+18  2e+00  2e+05
15:  1.7875e-03  1.7064e+23  1e+19  2e+00  9e+07
16:  1.7879e-03  8.9617e+26  8e+20  2e+00  6e+11
Terminated (singular KKT matri

In [304]:
ratio1 =  pd.Series(ratio, index = rx)

#Find a maximum Ratio
print("Mean-Variance Optimization is at: ",np.max(ratio1))
ratio1


Mean-Variance Optimization is at:  [[17216.31751156]]


0.000                   [[0.0]]
0.001     [[6.116674212558824]]
0.002    [[11.184020987073174]]
0.003     [[56.17846462889961]]
0.004    [[407.58500745634376]]
0.005     [[900.6842282868444]]
0.006    [[1052.5626968823428]]
0.007     [[667.2234651716292]]
0.008     [[324.9097390301945]]
0.009     [[36.00691753357427]]
0.010      [[40.5700837645482]]
0.011     [[48.75347440546539]]
0.012    [[14.971974136062828]]
0.013     [[52.39553984719872]]
0.014     [[24.64240745130365]]
0.015     [[8.959403009016665]]
0.016     [[48.98589876957388]]
0.017     [[6.987098582853241]]
0.018    [[37.998508801871175]]
0.019     [[32.45176034600611]]
0.020     [[62.88612877818493]]
0.021     [[43.49286461562307]]
0.022     [[36.94703158238759]]
0.023    [[3.2451068420393567]]
0.024    [[5.6792213424893365]]
0.025     [[3.628439267724399]]
0.026    [[3.4180630383174995]]
0.027    [[3.2297193104476754]]
0.028    [[3.0615550106182634]]
0.029      [[2.90922214619205]]
0.030     [[7.884452212528743]]
0.031   

In [296]:
#Q2

#Set number of assets and obersations
n_assets = 20
n_obs = 60
np.random.seed(2)
#generate random normal rates
return_vec = np.random.normal(0.025,0.015,(n_obs,n_assets))
return_vec = np.asmatrix(return_vec)
lamb = 0.1

#Create matrix for QP
P2 = opt.matrix(np.cov(return_vec, rowvar=False))
q2 = opt.matrix(1.0, (n_assets ,1))*lamb

#Merge G
G = -opt.matrix(np.eye(return_vec.shape[0],return_vec.shape[1]))
#Weight
Gw = np.transpose(opt.matrix(1.0, (n_assets ,1)))
G_sum2 = opt.matrix(np.concatenate((G,-Gw),axis=0))

#weight constraint

h = opt.matrix(0.0, (len(return_vec) + 1 ,1))


#Merge A
g = np.transpose(opt.matrix(1.0, (n_assets ,1)))
A2 = opt.matrix(g)

#Merge b interest rates and sum of weights = 1

b2 = opt.matrix(1.0)


#solution
sol1 = solvers.qp(-P2,q2,G_sum2,h,A2,b2)['x']


     pcost       dcost       gap    pres   dres
 0:  9.9995e-02 -2.8500e+00  1e+02  8e+00  2e+01
 1:  9.9995e-02 -1.9280e+00  3e+00  8e-02  2e-01
 2:  9.9995e-02  3.0255e-02  8e-02  8e-04  2e-03
 3:  9.9995e-02  9.9286e-02  8e-04  8e-06  2e-05
 4:  9.9995e-02  9.9976e-02  2e-05  2e-07  5e-07
Terminated (singular KKT matrix).


In [297]:
#Increasing Lambda
lamb = np.arange(0.0,2,0.1)
sol2 = []

#Use for loop to see how ratio of variance changes as Excess Return increases
for l in lamb:
    print(l)
    q2 = opt.matrix(1.0, (n_assets ,1))*l
    x_sol = solvers.qp(-P2,q2,G_sum2,h,A2,b2)['x']
    sol2.append(x_sol)
    

0.0
     pcost       dcost       gap    pres   dres
 0: -4.8588e-06 -2.9500e+00  1e+02  8e+00  2e+01
 1: -4.8582e-06 -2.0280e+00  3e+00  8e-02  2e-01
 2: -4.8582e-06 -6.9745e-02  8e-02  8e-04  2e-03
 3: -4.8592e-06 -7.1365e-04  8e-04  8e-06  2e-05
 4: -4.9585e-06 -2.3888e-05  2e-05  2e-07  5e-07
Terminated (singular KKT matrix).
0.1
     pcost       dcost       gap    pres   dres
 0:  9.9995e-02 -2.8500e+00  1e+02  8e+00  2e+01
 1:  9.9995e-02 -1.9280e+00  3e+00  8e-02  2e-01
 2:  9.9995e-02  3.0255e-02  8e-02  8e-04  2e-03
 3:  9.9995e-02  9.9286e-02  8e-04  8e-06  2e-05
 4:  9.9995e-02  9.9976e-02  2e-05  2e-07  5e-07
Terminated (singular KKT matrix).
0.2
     pcost       dcost       gap    pres   dres
 0:  2.0000e-01 -2.7500e+00  1e+02  8e+00  2e+01
 1:  2.0000e-01 -1.8280e+00  3e+00  8e-02  2e-01
 2:  2.0000e-01  1.3026e-01  8e-02  8e-04  2e-03
 3:  2.0000e-01  1.9929e-01  8e-04  8e-06  2e-05
 4:  2.0000e-01  1.9998e-01  2e-05  2e-07  5e-07
Terminated (singular KKT matrix).
0.30000

In [298]:
lambd =  pd.Series(sol2, index = lamb)

#Find a maximum for different Risk Aversion (lambdas)
print("Solution2 X for different value of lambdas: ", pd.Series(lambd))


Solution2 X for different value of lambdas:  0.0    [0.04934823782567344, 0.050409813508801006, 0....
0.1    [0.04934823782567349, 0.050409813508800014, 0....
0.2    [0.04934823782567423, 0.05040981350880131, 0.0...
0.3    [0.049348237825673484, 0.050409813508796454, 0...
0.4    [0.04934823782567354, 0.05040981350879637, 0.0...
0.5    [0.049348237825673553, 0.050409813508796766, 0...
0.6    [0.049348237825675156, 0.05040981350879757, 0....
0.7    [0.049348237825674414, 0.050409813508797426, 0...
0.8    [0.04934823782567475, 0.05040981350879783, 0.0...
0.9    [0.04934823782567473, 0.05040981350879734, 0.0...
1.0    [0.049348237825677384, 0.05040981350880014, 0....
1.1    [0.04934823782567554, 0.05040981350881465, 0.0...
1.2    [0.04934823782567535, 0.05040981350881447, 0.0...
1.3    [0.049348237825675524, 0.05040981350881491, 0....
1.4    [0.04934823782567515, 0.05040981350881483, 0.0...
1.5    [0.04934823782567557, 0.050409813508814856, 0....
1.6    [0.04934823782567509, 0.050409813508